In [ ]:
import torchvision
print(torchvision.__version__)

0.14.1+cu116


In [ ]:
#라이브러리, 하이퍼파라미터 설정
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np

# 하이퍼파라미터
EPOCHS = 500
BATCH_SIZE = 100
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("Using Device:", DEVICE)


Using Device: cpu


In [ ]:

# 학습에 필요한 데이터셋을 로딩합니다. 

# Fashion MNIST 데이터셋
trainset = datasets.FashionMNIST(
    './.data',
    train=True,
    download=True,
    transform=transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.5,), (0.5,))
    ])
)
train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = True
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw



In [ ]:
#GAN 학습
G = nn.Sequential(
        nn.Linear(64, 256),
        nn.ReLU(),
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 784),
        nn.Tanh()) # 결과 -1 or 1


# 판별자는 784차원의 텐서를 입력받음. 결과 : 진짜인지 예측값

# 판별자 (Discriminator)
D = nn.Sequential(
        nn.Linear(784, 256),
        nn.LeakyReLU(0.2), # 양의 기울기 or 음의 기울기 전달 
        nn.Linear(256, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 1),
        nn.Sigmoid())

# 모델의 가중치를 지정한 장치로 보내기
D = D.to(DEVICE)
G = G.to(DEVICE)

# 이진 크로스 엔트로피 (Binary cross entropy) 오차 함수와
# 생성자와 판별자를 최적화할 Adam 모듈
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G.parameters(), lr=0.0002)



total_step = len(train_loader)
for epoch in range(EPOCHS):
    for i, (images, _) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(DEVICE)
        
        # '진짜'와 '가짜' 레이블 생성
        real_labels = torch.ones(BATCH_SIZE, 1).to(DEVICE)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)
        
        # 판별자가 진짜 이미지를 진짜로 인식하는 오차를 예산
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # 무작위 텐서로 가짜 이미지 생성
        z = torch.randn(BATCH_SIZE, 64).to(DEVICE)
        fake_images = G(z)
        
        # 판별자가 가짜 이미지를 가짜로 인식하는 오차를 계산
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # 진짜와 가짜 이미지를 갖고 낸 오차를 더해서 판별자의 오차 계산
        d_loss = d_loss_real + d_loss_fake

        # 역전파 알고리즘으로 판별자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # 생성자가 판별자를 속였는지에 대한 오차를 계산
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)
        
        # 역전파 알고리즘으로 생성자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    # 학습 진행 알아보기
    print('Epoch [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
          .format(epoch, EPOCHS, d_loss.item(), g_loss.item(), 
                  real_score.mean().item(), fake_score.mean().item()))



Epoch [0/500], d_loss: 0.0253, g_loss: 5.2328, D(x): 0.99, D(G(z)): 0.02
Epoch [1/500], d_loss: 0.1170, g_loss: 4.3846, D(x): 0.95, D(G(z)): 0.01
Epoch [2/500], d_loss: 0.0999, g_loss: 6.6673, D(x): 0.99, D(G(z)): 0.02
Epoch [3/500], d_loss: 0.1064, g_loss: 6.9813, D(x): 0.97, D(G(z)): 0.02
Epoch [4/500], d_loss: 0.0861, g_loss: 5.4754, D(x): 0.96, D(G(z)): 0.02
Epoch [5/500], d_loss: 0.1141, g_loss: 5.0835, D(x): 0.95, D(G(z)): 0.02
Epoch [6/500], d_loss: 0.2576, g_loss: 3.4111, D(x): 0.98, D(G(z)): 0.18
Epoch [7/500], d_loss: 0.2115, g_loss: 6.2566, D(x): 0.95, D(G(z)): 0.04
Epoch [8/500], d_loss: 0.2345, g_loss: 5.0024, D(x): 0.94, D(G(z)): 0.06
Epoch [9/500], d_loss: 0.2548, g_loss: 4.4122, D(x): 0.95, D(G(z)): 0.10
Epoch [10/500], d_loss: 0.2970, g_loss: 3.8710, D(x): 0.91, D(G(z)): 0.04
Epoch [11/500], d_loss: 0.2169, g_loss: 4.2539, D(x): 0.92, D(G(z)): 0.04
Epoch [12/500], d_loss: 0.2095, g_loss: 4.4931, D(x): 0.92, D(G(z)): 0.03
Epoch [13/500], d_loss: 0.4592, g_loss: 5.7492, 

In [ ]:
# 생성자 결과물 시각화
z=torch.randn(BATCH_SIZE,64).to(DEVICE)
fake_images=G(z)

for i in range(10):
    fake_images_img=np.reshape(fake_images.data.cpu().numpy()[i],(28,28))
    plt.imshow(fake_images_img,cmap='gray')
    plt.show()